# **Introduction**
In this notebook we talk abount finetuning mbart-50 model on custom data for English to Urdu translation. We will walk through the complete process from preparing the data to feeding it into moodel.**We will be using 2 T4 GPUs for this purpose**

# **Data Preparation**
We will use "opus100" data corpus from huggingface for this purpose. The data consist of 753913 samples of text in both English and Urdu language. I first convert the data into csv format becasuse i am comfortable with handling csv's and i also take only 6000 samples (in which 5000 is for train 1000 is for validation) of data due to the time it takes for training the model.In the following code i have done all this step by step.

In [3]:
import datasets
import csv
import os

In [2]:
dataset = datasets.load_dataset("opus100",'en-ur')

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/753913 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset opus100 downloaded and prepared to /root/.cache/huggingface/datasets/opus100/en-ur/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
def save_translation_to_csv(input_dict, output_file):
    # Extract English and Urdu texts from the input dictionary
    english_text = input_dict['translation']['en']
    urdu_text = input_dict['translation']['ur']

    # Create a list of dictionaries containing the data to be saved in the CSV file
    data = [{'Urdu': urdu_text, 'English': english_text}]

    # Define the CSV file headers
    fieldnames = ['Urdu', 'English']

    # Check if the CSV file already exists
    file_exists = os.path.isfile(output_file)

    # Write the data to the CSV file
    with open(output_file, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write header only if the file is newly created
        if not file_exists:
            writer.writeheader()

        # Write the data row
        writer.writerows(data)


In [5]:
output_csv_file = 'translation.csv'
for n in range(6000):
   input_text=dataset['train'][n]
   save_translation_to_csv(input_text, output_csv_file)

# **Loading Data**
Now we load the data and prepare it to feed into model for that i have created 5 different style of prompts. By different style of prompts i mean i have defined 5 different ways in which the question is asked to the model and the answer is given by the model. As we know that every user try different ways to ask the same think so thats why i am defining 5 different ways you can change the prompts also increase the ways according to the portion of data you use

In [7]:
from datasets import concatenate_datasets,Dataset

In [2]:
#dataset = load_dataset("csebuetnlp/xlsum", "urdu")
#print(f"Features: {dataset['train'].column_names}")

In [8]:
import pandas as pd

In [9]:
dataset_file1 = '/kaggle/working/translation.csv'
dataset1 = pd.read_csv(dataset_file1)


In [10]:
dataset1

,Urdu,English
0,اورجب ہم نے موسیٰ سے چالیس رات کا وعدہ کیا پھر...,"Yet, remember, as We communed with Moses for f..."
1,"بجز تیرے اُن بندوں کے جنہیں تو نے خالص کر لیا ہے""","Except, among them, Your chosen servants."""
2,(اے پیغمبر لوگوں سے) کہہ دو کہ میرے پاس وحی آئ...,"Say, (O Prophet), it was revealed to me that a..."
3,ہم عنقریب اس کی ناک پر داغ لگائیں گے,Soon We shall brand him on his snout.
4,اور جو لوگ مومن مردوں اور عورتوں کو بے قصور اذ...,And those who unnecessarily harass Muslim men ...
...,...,...
5995,اور ہم نے انہیں بانٹ دیا بارہ قبیلے گروہ گروہ،...,We divided them into twelve tribal communities...
5996,جو تمہارا بھی پروردگار ہے اور تمہارے پہلے والے...,who is your Lord and the Lord of your forefath...
5997,تاکہ وہ اس کے پھل کھائیں اور اسے اُن کے ہاتھوں...,So that they may eat of its fruit; yet it was ...
5998,اتنے میں شہر کے دُور دراز گوشے سے ایک شخص دوڑت...,"Then there came running, from the farthest par..."


In [11]:
train_dataset=dataset1[:5000]
val_dataset=dataset1[5000:]

In [12]:
val_dataset.shape

(1000, 2)

Here we are dividing the 5000 train samples into 5 parts 1000 samples for a single type of prompt

In [13]:
data1=train_dataset[:1000]
data2=train_dataset[1000:2000]
data3=train_dataset[2000:3000]
data4=train_dataset[3000:4000]
data5=train_dataset[4000:5000]

Here i am creating a different dataset object and filling it by creating a prompt along with the data.Here engtext colomn is the source colomn and urdutext colomn is the target colomn

In [14]:
train_dataset1 = pd.DataFrame()
train_dataset1['engtext'] ='Can you translate "'+data1['English']+'" into urdu ?'
train_dataset1['urdutext']='Yes sure here is the urdu translation "'+data1['Urdu']+'".'
train_dataset2 = pd.DataFrame()
train_dataset2['engtext'] ='Please Translate "'+data2['English']+'" to urdu.'
train_dataset2['urdutext'] ='Okey the tranlation into urdu is "'+data2['Urdu']+'".I hope this will help.'
train_dataset3 = pd.DataFrame()
train_dataset3['engtext'] ='Please Translate "'+data3['English']+'" to urdu.'
train_dataset3['urdutext']="Yeah sure i can help you with that here is the translation in Urdu : "+data3['Urdu']+'.'
train_dataset4 = pd.DataFrame()
train_dataset4['engtext'] ='Can you translate "'+data4['English']+'" into urdu ?'
train_dataset4['urdutext']='Yes, the translation of "'+data4['English']+'" to Urdu is : "'+data4['Urdu']+'".'
train_dataset5 = pd.DataFrame()
train_dataset5['engtext'] ='Translate English text to Urdu. Here is the text :"'+data5['English']+'".'
train_dataset5['urdutext']='Sure, the translation of your given text to Urdu is : "'+data5['Urdu']+'".'
appended_dataframe = train_dataset1.append(train_dataset2, ignore_index=True)
appended_dataframe = appended_dataframe.append(train_dataset3, ignore_index=True)
appended_dataframe = appended_dataframe.append(train_dataset4, ignore_index=True)
appended_dataframe = appended_dataframe.append(train_dataset5, ignore_index=True)
train_data = Dataset.from_pandas(appended_dataframe)
train_data

/tmp/ipykernel_29/1214744637.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_dataframe = train_dataset1.append(train_dataset2, ignore_index=True)
/tmp/ipykernel_29/1214744637.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_dataframe = appended_dataframe.append(train_dataset3, ignore_index=True)
/tmp/ipykernel_29/1214744637.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_dataframe = appended_dataframe.append(train_dataset4, ignore_index=True)
/tmp/ipykernel_29/1214744637.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_dataframe = appended_dataframe.append(train_dataset5, ignore_inde

Dataset({
    features: ['engtext', 'urdutext'],
    num_rows: 5000
})

In [15]:
train_data['engtext'][3002]

'Can you translate "And they attribute to God what they themselves dislike, while their tongues utter the lie that theirs is the goodness. Without a doubt, for them is the Fire, and they will be neglected." into urdu ?'

In [16]:
train_data['urdutext'][3002]

'Yes, the translation of "And they attribute to God what they themselves dislike, while their tongues utter the lie that theirs is the goodness. Without a doubt, for them is the Fire, and they will be neglected." to Urdu is : "اور یہ خدا کے لیے ایسی چیزیں تجویز کرتے ہیں جن کو خود ناپسند کرتے ہیں اور زبان سے جھوٹ بکے جاتے ہیں کہ ان کو (قیامت کے دن) بھلائی (یعنی نجات) ہوگی۔ کچھ شک نہیں کہ ان کے لیے (دوزخ کی) آگ (تیار) ہے اور یہ (دوزخ میں) سب سے آگے بھیجے جائیں گے".'

In [17]:
data1=val_dataset[:200]
data2=val_dataset[200:400]
data3=val_dataset[400:600]
data4=val_dataset[600:800]
data5=val_dataset[800:1000]

In [18]:
val_dataset1 = pd.DataFrame()
val_dataset1['engtext'] ='Can you translate "'+data1['English']+'" into urdu ?'
val_dataset1['urdutext']='Yes sure here is the urdu translation "'+data1['Urdu']+'".'
val_dataset2 = pd.DataFrame()
val_dataset2['engtext'] ='Please Translate "'+data2['English']+'" to urdu.'
val_dataset2['urdutext'] ='Okey the tranlation into urdu is "'+data2['Urdu']+'".I hope this will help.'
val_dataset3 = pd.DataFrame()
val_dataset3['engtext'] ='Please Translate "'+data3['English']+'" to urdu.'
val_dataset3['urdutext']="Yeah sure i can help you with that here is the translation in Urdu : "+data3['Urdu']+'.'
val_dataset4 = pd.DataFrame()
val_dataset4['engtext'] ='Can you translate "'+data4['English']+'" into urdu ?'
val_dataset4['urdutext']='Yes, the translation of "'+data4['English']+'" to Urdu is : "'+data4['Urdu']+'".'
val_dataset5 = pd.DataFrame()
val_dataset5['engtext'] ='Translate English text to Urdu. Here is the text :"'+data5['English']+'".'
val_dataset5['urdutext']='Sure, the translation of your given text to Urdu is : "'+data5['Urdu']+'".'
appended_dataframe = val_dataset1.append(val_dataset2, ignore_index=True)
appended_dataframe = appended_dataframe.append(val_dataset3, ignore_index=True)
appended_dataframe = appended_dataframe.append(val_dataset4, ignore_index=True)
appended_dataframe = appended_dataframe.append(val_dataset5, ignore_index=True)
val_data = Dataset.from_pandas(appended_dataframe)
val_data

/tmp/ipykernel_29/2679250969.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_dataframe = val_dataset1.append(val_dataset2, ignore_index=True)
/tmp/ipykernel_29/2679250969.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_dataframe = appended_dataframe.append(val_dataset3, ignore_index=True)
/tmp/ipykernel_29/2679250969.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_dataframe = appended_dataframe.append(val_dataset4, ignore_index=True)
/tmp/ipykernel_29/2679250969.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_dataframe = appended_dataframe.append(val_dataset5, ignore_index=True)


Dataset({
    features: ['engtext', 'urdutext'],
    num_rows: 1000
})

In [19]:
val_data['engtext'][902]

'Translate English text to Urdu. Here is the text :"Yet if you were to ask them: “Who created the heavens and the earth?” they will certainly say: “The All-Mighty, the All-Knowing has created them.”".'

In [20]:
val_data['urdutext'][902]

'Sure, the translation of your given text to Urdu is : "اور آپ ان سے سوال کریں گے کہ زمین و آسمان کوکس نے پیدا کیا ہے تویقینا یہی کہیں گے کہ ایک زبردست طاقت والی اور ذی علم ہستی نے خلق کیاہے".'

# **Loading the model**

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer,MBartForConditionalGeneration

model_ckpt = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = MBartForConditionalGeneration.from_pretrained(model_ckpt)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Now we are going to create a function that takes the text data and convert it into input_ids and labels because as we know computer only understand numbers not text so we need to convert text into numbers for that we are using mbart tokenizer. Here max_length is set 1024 because that is the maximam tokens of string mbart can process at a time

In [22]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["engtext"], max_length=1024, padding="max_length",truncation=True)

    #with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch["urdutext"], max_length=1024, padding="max_length", truncation=True)

    return {"input_ids": input_encodings["input_ids"],
           "attention_mask": input_encodings["attention_mask"],
           "labels": target_encodings["input_ids"]}

train_dataset_tf = train_data.map(convert_examples_to_features, batched=True, remove_columns=["engtext","urdutext"])
val_dataset_tf = val_data.map(convert_examples_to_features, batched=True, remove_columns=["engtext","urdutext"])

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [23]:
train_dataset_tf

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

In [24]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [25]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer,TrainingArguments, Trainer

training_args = Seq2SeqTrainingArguments(output_dir='mbartTrans', num_train_epochs=2,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                evaluation_strategy='steps',save_strategy='no',eval_steps=2000,logging_steps=1000,
                                  weight_decay=0.01, push_to_hub=False,
                           fp16=True,learning_rate=2e-5,optim="adafactor")

In [26]:
import gc
gc.collect()

58

In [27]:
trainer = Seq2SeqTrainer(model=model, args=training_args, tokenizer=tokenizer,
                  data_collator=seq2seq_data_collator,
                  train_dataset=train_dataset_tf,
                        eval_dataset=val_dataset_tf)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
2000,0.106500,0.098222
4000,0.078100,0.092087


TrainOutput(global_step=5000, training_loss=0.21711170501708985, metrics={'train_runtime': 12166.1658, 'train_samples_per_second': 0.822, 'train_steps_per_second': 0.411, 'total_flos': 2.167129767936e+16, 'train_loss': 0.21711170501708985, 'epoch': 2.0})

In [28]:
trainer.save_model("mbartfortranslation5000")

In [29]:
import shutil
shutil.make_archive('mbartfortranslation', 'zip', '/kaggle/working/mbartfortranslation5000')

'/kaggle/working/mbartfortranslation.zip'

> # **Testing the model on custom prompts**

Note: Restart and clear cell output to release gpus memory because we are loading the model on gpu for generating faster responces. Save the notebook sos that you can not lose your work

In [29]:
from transformers import MBartForConditionalGeneration, AutoTokenizer,pipeline

In [32]:
model_ckpt = "/kaggle/working/bloomzfortran"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForCausalLM.from_pretrained("/kaggle/working/bloomzfortran").to('cuda:0').half()

In [72]:
prompt1='Please Translate "How are you." to urdu.'
prompt1

'Please Translate "How are you." to urdu.'

In [74]:
generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer,device='cuda:0',do_sample=False)

In [ ]:
result = generator(prompt, max_length=1024)

In [75]:
result

[{'generated_text': 'Please Translate "How are you." to urdu.Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Okey the tranlation into urdu is "کیا تم ٹھیک ہیں؟".Oke